# text2rdf

## Imports

In [ ]:
# General purpose
import os
import glob
import random
import xml.etree.ElementTree as ET

# PyTorch 
import torch
import torch.nn as nn
from torch import optim

# Bert
from transformers import BertTokenizer, BertModel, BertConfig
#Bleu score
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
device = torch.device('cuda')
print("Device:", device)

## Preprocessing

### Select Data-sub-set (given restricted nr of triples per sentence)

In [ ]:
# How many triples to train and test system on (min: 1, max: 7)
MIN_NUM_TRIPLES = 1
MAX_NUM_TRIPLES = 2

In [ ]:
# Set paths where to retrieve data from
DS_BASE_PATH = './WebNLG/'

TRAIN_PATH = DS_BASE_PATH + 'train/'
TEST_PATH = DS_BASE_PATH + 'dev/'

TRAIN_DIRS = [ TRAIN_PATH + str(i) + 'triples/' for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1) ]
TEST_DIRS  = [ TEST_PATH  + str(i) + 'triples/' for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1) ]

In [ ]:
# Print selected directories
print('Train dirs:', TRAIN_DIRS)
print('Test  dirs:', TEST_DIRS)

### Load Data

#### Load Settings (do not touch)

In [ ]:
originaltripleset_index = 0  # Index of 'originaltripleset' attribute in XML entry
modifiedtripleset_index = 1  # Index of 'modifiedtripleset' attribute in XML entry
first_lexical_index = 2      # Index as of which verbalizations of RDF triples start in entry

#### Train Data

In [ ]:
# Usage of train: train[target_nr_triples][element_id]['target_attribute']
train = [[] for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Documents how many entries there are per number of triples
train_stats = [0 for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Iterate through all files per number of triples and per category and load data
for i, d in enumerate(TRAIN_DIRS):
    nr_triples = list(range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1))[i]
    
    for filename in glob.iglob(d+'/**', recursive=False):
        if os.path.isfile(filename): # Filter dirs
            #print('File:', filename)
            
            tree = ET.parse(filename)
            root = tree.getroot()
            
            entries = root[0]
            train_stats[nr_triples-MIN_NUM_TRIPLES] += len(entries)
            
            for entry in entries:
                #print('Original triple set: ', entry[originaltripleset_index])
                #print('Modified triple set: ', entry[modifiedtripleset_index])
                
                modified_triple_set = entry[modifiedtripleset_index]
                unified_triple_set = []
                
                for triple in modified_triple_set:
                    # Make a list containing a conjunction of all individual triples
                    triple_list = [x.strip() for x in triple.text.split('|')]
                    unified_triple_set += triple_list
                    
                verbalizations = entry[first_lexical_index:]

                for verbalization in verbalizations:
                    if verbalization.text.strip() == '':
                        continue
                    #print('Text:', verbalization, verbalization.tag, verbalization.attrib, verbalization.text)
                    #print('Trip:', triple, triple.tag, triple.attrib, triple.text)

                    train[i].append({ 'category': entry.attrib['category'],
                                      'id': entry.attrib['eid'],
                                      'triple_cnt': nr_triples,
                                      'text': verbalization.text,
                                      'triple': unified_triple_set,
                                    })
                        
print(train)
print(train_stats)

#### Test Data

In [ ]:
# Usage of test: test[target_nr_triples][element_id]['target_attribute']
test = [[] for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Documents how many entries there are per number of triples
test_stats = [0 for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Iterate through all files per number of triples and per category and load data
for i, d in enumerate(TEST_DIRS):
    nr_triples = list(range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1))[i]
    
    for filename in glob.iglob(d+'/**', recursive=False):
        if os.path.isfile(filename): # Filter dirs
            #print('File:', filename)
            
            tree = ET.parse(filename)
            root = tree.getroot()
            
            entries = root[0]
            test_stats[nr_triples-MIN_NUM_TRIPLES] += len(entries)
            
            for entry in entries:
                #print('Original triple set: ', entry[originaltripleset_index])
                #print('Modified triple set: ', entry[modifiedtripleset_index])
                
                modified_triple_set = entry[modifiedtripleset_index]
                unified_triple_set = []
                
                for triple in modified_triple_set:
                    # Make a list containing a conjunction of all individual triples
                    triple_list = [x.strip() for x in triple.text.split('|')]
                    unified_triple_set += triple_list
                    
                verbalizations = entry[first_lexical_index:]

                for verbalization in verbalizations:
                    if verbalization.text.strip() == '':
                        continue
                    #print('Text:', verbalization, verbalization.tag, verbalization.attrib, verbalization.text)
                    #print('Trip:', triple, triple.tag, triple.attrib, triple.text)

                    test[i].append({ 'category': entry.attrib['category'],
                                      'id': entry.attrib['eid'],
                                      'triple_cnt': nr_triples,
                                      'text': verbalization.text,
                                      'triple': unified_triple_set,
                                    })
                        
print(test)
print(test_stats)

#### Spilt Train Data into Train and Dev (for intermindiate validation throughout training)

In [ ]:
# Percentage of train data reserved for validation throughout training
dev_percentage = 0.15

In [ ]:
# Init dev dataset
dev = [[] for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Sample number of dev instances per number of triples
dev_stats = [int(dev_percentage * train_stats[i]) for i in range(0, MAX_NUM_TRIPLES+1-MIN_NUM_TRIPLES)]

print('Samples per nr of triples:', dev_stats)

# Sample indices to be reserved for dev dataset for each nr of triples
dev_indices = [random.sample(range(0, len(train[i])), dev_stats[i]) for i in range(0, MAX_NUM_TRIPLES+1-MIN_NUM_TRIPLES)]
for i in range(len(dev_indices)):
    dev_indices[i].sort(reverse=True)

# Copy selected dev-entries into dev & delete all duplicates/related entries from train dataset
for nr_triples in range(0, MAX_NUM_TRIPLES+1-MIN_NUM_TRIPLES):
    
    # Iterate through all indices reserved for validation set (per nr of triples)
    for index in dev_indices[nr_triples]:
        
        # Select index'th train entry (to become dev/validation data)
        selected_entry = train[nr_triples][index]
        
        # Extract indentifying attributes
        entrys_category = selected_entry['category']
        entrys_idx = selected_entry['id']
        
        # Put selected entry into dev set
        dev[nr_triples].append(selected_entry)
        
        # Find all entries of matching index & category and remove them from train data
        for entry in train[nr_triples]:
            if entry['id'] == entrys_idx and entry['category'] == entrys_category:
                train[nr_triples].remove(entry)
                
print(dev)
print(dev_stats)

#### Print Stats

In [ ]:
print('Minimal number of triples:', MIN_NUM_TRIPLES)
print('Maximal number of triples:', MAX_NUM_TRIPLES)

print()

print('Training: ')
for nr_triples in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1):
    print('Given %i triples per sentence: ' % nr_triples)
    print('Number of combinations of triples and verbalizations:', len(train[nr_triples-MIN_NUM_TRIPLES]))

print()

print('Dev: ')
for nr_triples in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1):
    print('Given %i triples per sentence: ' % nr_triples)
    print('Number of combinations of triples and verbalizations:', len(dev[nr_triples-MIN_NUM_TRIPLES]))

print()

print('Testing: ')
for nr_triples in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1):
    print('Given %i triples per sentence: ' % nr_triples)
    print('Number of combinations of triples and verbalizations:', len(test[nr_triples-MIN_NUM_TRIPLES]))

# Neural Machine Translation (NMT) Model Definition 

## TODO: needs updating

## Idea:
1. Encoder: 
1.1 Input==Word Embedding; 
1.2 Output==Context Vector (that is: Encoding of sentence; contained in hidden state after having observed last embedding)

2. Decoder:
2.1 Input==Context Vector
2.2 Output==Probability distribution over output vocab

3. Seq2Seq model: Combining the two

## BERT Encoder

#### Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', device=device)

#### Model

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True).to(device)

## Decoder

### Soft Attention Model
This model implements the Soft Attention model presented in http://proceedings.mlr.press/v37/xuc15.pdf. 
1. Attention energies (i.e. energy per annotation vector) get computed: $e_{ti}=f_{att}(a_i,h_{t−1})$. Note that this formula implies that the Decoder's previous hidden state $h_{t-1}$ needs to be appended to each individual annotation vector $a_i$ before feeding their concatenation through a fully-connected layer $f_{att}$. 
2. Attention weights $\alpha$ get computed from the aforementioned energies: $\alpha_t = softmax(e_t)$, where $\alpha_{ti} = \frac{exp(e_{ti})}{\sum^L_{k=1} exp(e_{tk})}$.

Note: $t$ stands for time, while $i$ identifies the particular annotation vector currently under consideration.

In [ ]:
class SoftAttention(nn.Module):
    
    def __init__(self, 
                 annotation_size,  # Tuple: (num_annotations, num_features_per_annotation)
                 hidden_len        # Number of nodes in Decoder's hidden state weight matrix
                ):
        
        super(SoftAttention, self).__init__()
        #print('SA INIT')
        # Variables
        self.num_annotations = annotation_size[0]
        self.annotation_features = annotation_size[1]
        self.hidden_size = hidden_len
        
        # Layers
        self.attn = nn.Linear(self.annotation_features + self.hidden_size, 1, bias=True)
        self.softmax = nn.Softmax(dim=1)
        #print('an size:', annotation_size) # 8x96
        #print('an features (96?):', self.annotation_features) # 96
        #print('hid size:', hidden_len)     # 96
        
    def forward(self, annotations, prev_hidden):
        
        # Repeat prev_hidded X times to append it to each of the annotation vectors (per batch element)
        repeated_hidden = torch.cat(
            [
                torch.repeat_interleave(hid, repeats=self.num_annotations, dim=0).unsqueeze(0)
                for hid in prev_hidden.split(1)
            ]
        )
        
        # Append previous hidden state to all annotation vectors (for each individual batch element)
        # Input to attention weight calculation
        #print('SA:', annotations.size(), repeated_hidden.size())
        input = torch.cat((annotations, repeated_hidden), dim=2)
        #print('Input size:', input.size())
        #print(self.attn)
        
        # Compute the relative attention scores per feaure (e_{ti}=f_{att}(a_i,h_{t−1}) from paper)
        energies = self.attn(input)
        
        #print('energies...')
        
        # Compute final attention weights (i.e. alpha)
        attn_weights = self.softmax(energies)
        #print('attn_weights:', attn_weights.size())
        return attn_weights


### Decoder itself (employing Soft Attention)

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, 
                 annotation_size,      # Size of annotation vectors produced by Encoder
                 out_vocab_size,       # How many words there are in the RDF-output language
                 embedding_dim,        # Length of a word embedding
                 hidden_dim,           # Nr hidden nodes
                 output_dim,           # Vocab size
                 bidirectional=False,  # Whether to have bidirectional GRU
                 n_layers=1,           # Nr layers in GRU
                 drop_prob=0.2         # Percent of node-dropouts
                ):
        
        super(Decoder, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.n_directions = 1 if not bidirectional else 2  # TODO: make use of it...
        
        self.attn = SoftAttention(annotation_size=annotation_size, hidden_len=hidden_dim)
        self.gru = nn.GRU(annotation_size[1]+embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, 
                annotations,  # Static annotation vectors (for each batch element)
                embeddings,   # Word embeddings of most recently generated word (per batch element)
                h_old         # Previous hidden state per batch element
               ):
        #print('Decoder forward:')
        #print('embeddings:\t', embeddings.size())
        #print('h_old:\t\t', h_old.size())
        
        annotation_weights = self.attn(annotations, h_old.squeeze())#.unsqueeze(2)
        #print('annotations:', annotations.size())
        #print('annotation_weights:', annotation_weights.size())
        weighted_annotations = annotations * annotation_weights
        #print('weighted_annotations:', weighted_annotations.size())
        context_vectors = torch.sum(weighted_annotations, dim=1)
        #print('context_vectors:', context_vectors.size())
        
        x = torch.cat((context_vectors, embeddings), dim=1)
        #print('x:', x.size())
        x = x.unsqueeze(1) # Add une dimension for 'sequence'
        
        #print('Decoder x:', x.size(), 'h_old:', h_old.size())
        #print(self.gru)
        out, h = self.gru(x, h_old)
        out = out.squeeze()
        out = self.softmax(self.fc(self.relu(out)))
        #print('h:', h.size())
        return out, h
    
    
    def init_hidden(self, annotation_vectors):
        # Mean of annotation vector per batch element
        # Assumes that number of hidden nodes == number annotation features
        hidden = torch.mean(annotation_vectors, dim=1)#.to(device)
        return hidden

## Encoder for seq2seq

In [ ]:
# use it only if required otherwise can be removed.


class Encoder(nn.Module):
    
    def __init__(self, 
                 input_dim,
                 embedding_dim,        # Length of a word embedding
                 hidden_dim,           # Nr hidden nodes
                 n_layers = 1,         # Nr layers in GRU
                 dropout =0.2
                ):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):        
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim] 
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs =
        #hidden =
        #cell =
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

## Train loop

In [ ]:
def predict(x, 
            word_embeddings,        # Decoder's word embeddings
            word2idx,               # 
            idx2word,               # 
            encoder,                # 
            decoder,                # 
            tokenizer,              # 
            loss_fn,                # 
            max_len=7,              # 
            batch_size=32,          # 
            compute_grads=False,    # 
            targets=None,           # 
            return_textual=False    # Whether to return predictions in index-form (default) or as textual strings
           ):
    
    print('In predict:')
    
    accumulated_loss = 0.
    
    # Init documentation of predictions
    predicted_indices = torch.zeros([batch_size, max_len]).to(device) # Numeric
    if return_textual:
        predicted_words = ['']*batch_size
    
    # Tokenize sampled minibatch sentences
    inputs = tokenizer(x, 
                       return_tensors="pt",     # Return tensors in pt=PyTorch format
                       padding=True,            # Pad all sentences in mini-batch to have the same length
                       add_special_tokens=True).to(device) # Add "Start of sequence", "End of sequence", ... tokens. 
    
    #print('Tokenized Inputs:', inputs)
    
    # Encode sentences: Pass tokenization output-dict-contents to model
    outputs = encoder(**inputs)
    #print('Got outputs:', outputs)

    # Retrieve hidden state to be passed into Decoder as annotation vectors
    # Reshape to get a set of 8 feature vectors from last hidden state
    annotations = outputs.last_hidden_state[:, -1, :].reshape(batch_size,8,-1).to(device)
    #print('Annotations size after cropping & reshape:', annotations.size())

    # Init Decoder's hidden state
    hidden = decoder.init_hidden(annotations).unsqueeze(0).to(device)
    #print('Initial hidden size:', hidden.size(), 'given annotations:', annotations.size())
    
    # Construct initial embeddings (start tokens)
    embeddings = word_embeddings(torch.zeros([batch_size], dtype=int).to(device)).to(device)
    
    for t in range(max_len):
        #print('START OF ITERATION', t)
        # Get decodings (aka prob distrib. over output vocab per batch element) for time step t
        prob_dist, hidden = decoder(annotations, # Static vector containing annotations per batch element 
                                    embeddings,  # Word embedding predicted last iteration (per batch element)
                                    hidden       # Decoder's hidden state of last iteratipn per batch element
                                    )

        # Get predicted word index from predicted probability distribution (per batch element)
        word_index = torch.max(prob_dist, dim=1).indices
        #print('Predicted word indices batch:', word_index)
        
        # Get corresponding word embedding (by index; per batch element)
        embedding = word_embeddings(word_index.to(device))
        
        # TODO: optional teacher forcing?

        # Record predicted words
        predicted_indices[:, t] = word_index
        #print('Predicted indices:', predicted_indices)
        
        # Record textual words if required
        if return_textual:
            
            # Get predicted word (per batch element)
            predicted_word = [idx2word(batch_element) for batch_element in word_index]
        
            for e in range(batch_size):
                predicted_words[e] += predicted_word[e]

        if compute_grads:
            
            #print('prob_dist:', prob_dist.size())
            #print('targets:', targets[:, t].size(), targets[:, t])
            
            # Compute (averaged over all batch elements given current time step t)
            loss = loss_fn(prob_dist, targets[:, t]).to(device)

            # Compute & back-propagate gradients
            loss.backward(retain_graph=True)
            
            # Document loss
            accumulated_loss += loss.item()
        #print('END OF ITERATION', t)
            
    ret_object = {
        'predicted_indices': predicted_indices,
    }
    
    print('Targets:\n', targets)
    print('Predicted idxs:\n', predicted_indices)
    
    if compute_grads:
        ret_object['loss'] = accumulated_loss
        #print('Accumulated loss:', accumulated_loss)
    if return_textual:
        ret_object['predicted_words'] = predicted_words
        #print("Predicted words:", predicted_words)
    #print("Returning from predict")
    return ret_object 

In [ ]:
def rdf_vocab_constructor(raw_vocab):
    #print(raw_vocab)
    vocab_count, word2idx, idx2word = 3, {'START': 0, 'PAD': 1, 'END': 2}, {0: 'START', 1: 'PAD', 2: 'END'}
    
    for partition in raw_vocab: # Different partitions with respect to nr or triples per sentence
        for train_instance in partition:
            triple = train_instance['triple']
            for token in triple:
                if token not in word2idx:
                    word2idx[token] = vocab_count
                    idx2word[vocab_count] = token
                    vocab_count += 1
    return vocab_count, word2idx, idx2word

In [ ]:
def training(train_data, 
          val_data,  
          epochs, 
          minibatch_size=32,
          embedding_dim=300,
          eval_frequency=10, # Every how many epochs to run intermediate evaluation
          learning_rate=0.0001
         ):
    
    # Construct RDF vocab
    vocab_count, word2idx, idx2word = rdf_vocab_constructor(train_data)
    
    # Construct embeddings
    rdf_vocab = nn.Embedding(num_embeddings=vocab_count, embedding_dim=embedding_dim, padding_idx=0).to(device)
    
    # Define model
    encoder = bert_model.to(device)
    decoder = Decoder(
        annotation_size=(8,96),    # Size of annotation vectors produced by Encoder
        out_vocab_size=vocab_count, # How many words there are in the RDF-output language
        embedding_dim=300,          # Length of a word embedding
        hidden_dim=96,             # Nr hidden nodes
        output_dim=vocab_count,             # Vocab size
    ).to(device)
    
    # Optimizer
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    loss = nn.CrossEntropyLoss()
    
    # For both train and validation data & for all number of tuples per sentence 
    # (in [MIN_NUM_TRIPLES, MAX_NUM_TRIPLES]), get the nr of train-/test instances
    len_x_train = [len(train_set) for train_set in train]
    len_x_val = [len(val_set) for val_set in dev]
    
    # Development of both train- and validation losses over course of training
    train_losses, val_losses = [0.]*epochs, [0.]*epochs
    
    print('Starting training.')
    
    # Train
    for epoch in range(epochs):
        print('Epoch:', epoch)
        
        train_loss, eval_loss = 0., 0.
        
        # Reset gradients
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        # Perform own train step for each nr of triples per sentence separately
        for i, nt in enumerate(range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)):
            print(str(i) + '. Condition:', nt, 'triples per sentence.')
            
            # Sample minibatch indices
            minibatch_idx = random.sample(population=range(len_x_train[i]), k=minibatch_size)
            #print('MB indices:', minibatch_idx)
            
            # Number of tokens to be predicted (per batch element)
            num_preds = nt*3+1 # = nr triples * 3 + stop_token 
            #print('Number of predictions:', num_preds)
            
            # Construct proper minibatch
            inputs = [train_data[i][idx]['text'] for idx in minibatch_idx]
            targets = torch.ones([minibatch_size, num_preds], dtype=int).to(device)
            
            #print('Inputs:', inputs)
            #print('Targets:', targets)
            
            for mb_i, idx in enumerate(minibatch_idx):
                #print('Text:', train_data[i][idx]['text'])
                #print('Triple:', train_data[i][idx]['triple'])
                for t, token in enumerate(train_data[i][idx]['triple']):
                    targets[mb_i, t] = word2idx[token]
            targets[:, -1] = 2  # 2 = Stop word index
            
            #print('Processed targets:', targets)
            #print('Predicting:')
            
            # Predict
            ret_object = predict(inputs,
                                 rdf_vocab,              # Decoder's word embeddings
                                 word2idx,               # 
                                 idx2word,               # 
                                 encoder,                # 
                                 decoder,                # 
                                 tokenizer,              # 
                                 loss,                   # 
                                 max_len=num_preds,      # Nr of tokens to be predicted
                                 batch_size=32,          # 
                                 compute_grads=True,     # 
                                 targets=targets,        # 
                                 return_textual=False    # Whether to return predictions in index-form (default) or as textual strings
                                )
            
            print('Return object:', ret_object)
            train_loss += ret_object['loss']
            #print("Returned loss:", ret_object['loss'])
            
        # Apply gradients
        encoder_optimizer.step()
        decoder_optimizer.step()
        #print('Optimizations performed.')
        
        # Intermediate evaluation
        
        # Save losses
        train_losses[epoch] = train_loss
        
    return train_losses, val_losses, encoder, decoder


# Train

In [ ]:
train_losses, val_losses, encoder, decoder = training(train, dev, epochs=100, learning_rate=0.0001)
print('Train losses:', train_losses)

# Test
### Used exclusively for evaluation on test data after training is fuly finished

## Evaluation - Bleu Score

In [ ]:
# Function for calculating the BLEU score for multiple sentence.
def calculate_bleu(data, train, dev, model, max_len = 7):
    
    trgs = []
    pred_trgs = []       
    src = dev
    trg = test
    # Get the data and feed it into pred_trg after Seq2seq
    #pred_trg = pred_trg[:-1]      
    #pred_trgs.append(pred_trg)
    #trgs.append([trg])
        
    return corpus_bleu(pred_trgs, trgs)